### Script to run all prompts on the data set ###

In [1]:
import os
import pandas as pd
import numpy as np
import logging
import time

logger = logging.getLogger(__name__)

# Import this module with autoreload
%load_ext autoreload
%autoreload 2
import llmt
from llmt.llmtools import Prompt
from llmt.llmtools import MentalHealth, OutpatientServices, InpatientServices
from llmt.llmtools import process_prompt
from llmt.openai import OpenAI, create_messages
from llmt.performance import Performance

In [2]:
# Parameters
model = 'gpt-4o'
temperature = 0

# Directories and files
data_dir = os.path.join(os.environ.get('HOME'), 'home_data', 'hcp')
test_file_name = 'hcp-alldata-250413.parquet'
test_file = os.path.join(data_dir, test_file_name)
df_all = pd.read_parquet(test_file)
# Filter the labeled data
df_train = df_all.loc[df_all['dset'] == 'train'].\
                reset_index(drop=True).\
                astype({'mental_health': int,
                        'inpatient': int,
                        'outpatient': int})
display(df_train.head())
print(df_train.shape)

,id,name,description,mental_health,inpatient,outpatient,dset
0,431643-07,Actriv,Provider of healthcare staffing services based...,2,0,0,train
1,310749-31,Alima,Operator of a non-governmental organization in...,0,0,0,train
2,162054-28,Apothecare,Provider of pharmacy services intended to prov...,0,0,0,train
3,597285-28,April Health (Clinics/Outpatient Services),Provider of mental health services intended to...,1,0,1,train
4,373978-90,Arise Child and Family Service,Operator of independent living centers caterin...,2,0,0,train


(187, 7)


In [3]:
# Add some additional samples 
test_samples = 10
random_state = 111
df_test = df_all.loc[df_all['mental_health'].isnull()].\
                sample(n=test_samples, replace=False, random_state=random_state).\
                reset_index(drop=True)
# Combine the training and test samples
df = pd.concat([df_train, df_test], axis=0, ignore_index=True).\
                sample(frac=1, random_state=random_state).\
                reset_index(drop=True)
for dset in ['train', 'test']:
    print(f'{dset}: {len(df.loc[df['dset']==dset, 'id'].unique())}')

display(df.head())

train: 187
test: 10


,id,name,description,mental_health,inpatient,outpatient,dset
0,157836-97,Assurance Health System,Operator of an inpatient psychiatric hospital ...,1.0,1.0,2.0,train
1,54831-97,Wellpath Care,Operator of a healthcare company intended to p...,1.0,0.0,2.0,train
2,82787-86,American Addiction Centers,Provider of inpatient substance abuse treatmen...,1.0,1.0,2.0,train
3,41318-92,Comprehensive Behavioral Care,Advanzeon Solutions Inc is engaged in the prov...,NaN,NaN,NaN,test
4,234726-22,Headway (Clinics/Outpatient Services),Operator of a mental healthcare system intende...,1.0,0.0,2.0,train


In [4]:
def predict_mh(name, description, version, model='gpt-4o', client=None, temperature=0): 
    variable = 'mental_health'
    pred_col = 'pred_mh'
    prompt_name = f'{variable}_system_{str(version).zfill(2)}'
    system_prompt = Prompt().load(prompt_name=prompt_name)
    user_prompt = process_prompt(f"""
                    The organization {name} is described as: {description} 
                    Does this organization provide {variable} healthcare services?
                    """)
    messages = create_messages(system_prompt=system_prompt, user_prompt=user_prompt)
    output = OpenAI().send_messages(messages=messages,
                                    model=model,
                                    temperature=temperature,
                                    response_format=MentalHealth,
                                    client=client)
    # Replace the boolean fields with binary outcome prediction
    output.update({pred_col: 1 if output.get(pred_col) == True else 0})
    # Select the fields that we want 
    output = {pred_col: output.get(pred_col)}
    return output

def predict_ip(name, description, version, model='gpt-4o', client=None, temperature=0): 
    variable = 'inpatient'
    pred_col = 'pred_ip'
    prompt_name = f'{variable}_system_{str(version).zfill(2)}'
    system_prompt = Prompt().load(prompt_name=prompt_name)
    user_prompt = process_prompt(f"""
                    The organization {name} is described as: {description} 
                    Does this organization provide {variable} healthcare services?
                    """)
    messages = create_messages(system_prompt=system_prompt, user_prompt=user_prompt)
    output = OpenAI().send_messages(messages=messages,
                                    model=model,
                                    temperature=temperature,
                                    response_format=InpatientServices,
                                    client=client)
    # Replace the boolean fields with binary outcome prediction
    output.update({pred_col: 1 if output.get(pred_col) == True else 0})
    # Select the fields that we want 
    output = {pred_col: output.get(pred_col)}
    return output

def predict_op(name, description, version, model='gpt-4o', client=None, temperature=0): 
    variable = 'outpatient'
    pred_col = 'pred_op'
    prompt_name = f'{variable}_system_{str(version).zfill(2)}'
    system_prompt = Prompt().load(prompt_name=prompt_name)
    user_prompt = process_prompt(f"""
                    The organization {name} is described as: {description} 
                    Does this organization provide {variable} healthcare services?
                    """)
    messages = create_messages(system_prompt=system_prompt, user_prompt=user_prompt)
    output = OpenAI().send_messages(messages=messages,
                                    model=model,
                                    temperature=temperature,
                                    response_format=OutpatientServices,
                                    client=client)
    # Replace the boolean fields with binary outcome prediction
    key_list = [pred_col, 'verified_op']
    output.update({key: 1 if output.get(key) == True else 0 for key in key_list})
    # Select the fields that we want 
    output = {pred_col: output.get(pred_col), 'verified_op': output.get('verified_op')}
    return output

In [5]:
# Try this function
company_id_list = sorted(list(df['id'].unique()))
company_id = company_id_list[10]
df_id = df.loc[df['id'] == company_id]
name = df_id['name'].values[0]
description = df_id['description'].values[0]
client = OpenAI().create_client()

model_params = {'model': 'gpt-4o', 
                'client': client, 
                'temperature': 0}

response_mh = predict_mh(name=name, description=description, version=2, **model_params)
print(response_mh)
response_ip = predict_ip(name=name, description=description, version=1, **model_params)
print(response_ip)
response_op = predict_op(name=name, description=description, version=1, **model_params)
print(response_op)

{'pred_mh': 1}
{'pred_ip': 1}
{'pred_op': 1, 'verified_op': 1}


In [11]:
client = OpenAI().create_client()
model_params = {'model': 'gpt-4o', 
                'client': client, 
                'temperature': 0}

runs = 10

# Run the prompt on all data
company_id_list = sorted(list(df['id'].unique()))
results_df_list = []
start_time = time.perf_counter()

for run in range(runs):
    execution_time = time.perf_counter() - start_time
    execution_time_min = np.round(execution_time/60, decimals=1)
    print(f'Execution time: {execution_time_min} minutes.')
    print(f'STARTING RUN {run + 1} / {runs}')
    for c, company_id in enumerate(company_id_list):
        if (c + 1) % 20 == 0:
            print(f'Sending description {c + 1} / {len(company_id_list)} to the model')
    
        df_id = df.loc[df['id'] == company_id]
        name = df_id['name'].values[0]
        description = df_id['description'].values[0]
        
        # Mental health predictions
        response_mh = predict_mh(name=name, description=description, version=2, **model_params)
        df_id = df_id.assign(**response_mh)
        
        # Inpatient predictions
        response_ip = predict_ip(name=name, description=description, version=1, **model_params)
        df_id = df_id.assign(**response_ip)
        
        # Outpatient predictions
        response_op = predict_op(name=name, description=description, version=1, **model_params)
        df_id = df_id.assign(**response_op)

        # Add the new data frame to the list
        df_id = df_id.assign(run=run, 
                             temp=model_params.get('temperature'))
        results_df_list.append(df_id)

results_df = pd.concat(results_df_list, axis=0, ignore_index=True)

# Save the results
results_file_name = f'250420_results_10_runs.parquet'
results_file = os.path.join(data_dir, results_file_name)
results_df.to_parquet(results_file)
print(results_file)
end_time = time.perf_counter()
execution_time = end_time - start_time
execution_time_min = np.round(execution_time/60, decimals=1)
print(f'Execution time: {execution_time_min} minutes.')

Execution time: 0.0 minutes.
STARTING RUN 1 / 10
Sending description 20 / 197 to the model
Sending description 40 / 197 to the model
Sending description 60 / 197 to the model
Sending description 80 / 197 to the model
Sending description 100 / 197 to the model
Sending description 120 / 197 to the model
Sending description 140 / 197 to the model
Sending description 160 / 197 to the model
Sending description 180 / 197 to the model
Execution time: 5.2 minutes.
STARTING RUN 2 / 10
Sending description 20 / 197 to the model
Sending description 40 / 197 to the model
Sending description 60 / 197 to the model
Sending description 80 / 197 to the model
Sending description 100 / 197 to the model
Sending description 120 / 197 to the model
Sending description 140 / 197 to the model
Sending description 160 / 197 to the model
Sending description 180 / 197 to the model
Execution time: 10.3 minutes.
STARTING RUN 3 / 10
Sending description 20 / 197 to the model
Sending description 40 / 197 to the model
Se

In [13]:
# Load the data and check the results
# results_file_name = 'inpatient_01_results.parquet'
results_file_name = f'250420_results_10_runs.parquet'
results_file = os.path.join(data_dir, results_file_name)
df = pd.read_parquet(results_file)

results_train = df.loc[df['dset'] == 'train']

# Mapping true variable and predictions
variable_dict = {'mental_health': 'pred_mh',
                 'inpatient': 'pred_ip',
                 'outpatient': 'pred_op'}

for true_col in variable_dict.keys():
    print(f'PERFORMANCE: {true_col.upper()}')
    display(Performance(data=df.copy()).\
            binary_performance(true_col=true_col, pred_col=variable_dict.get(true_col)))
    print()

PERFORMANCE: MENTAL_HEALTH


{'p': 1360,
 'n': 350,
 'tp': 942,
 'tn': 307,
 'fp': 43,
 'fn': 418,
 'recall': 0.6926,
 'precision': 0.9563,
 'min_precision': 0.7953,
 'specificity': 0.8771,
 'f_score': 0.8034}


PERFORMANCE: INPATIENT


{'p': 690,
 'n': 1130,
 'tp': 680,
 'tn': 1033,
 'fp': 97,
 'fn': 10,
 'recall': 0.9855,
 'precision': 0.8752,
 'min_precision': 0.3791,
 'specificity': 0.9142,
 'f_score': 0.9271}


PERFORMANCE: OUTPATIENT


{'p': 940,
 'n': 420,
 'tp': 771,
 'tn': 309,
 'fp': 111,
 'fn': 169,
 'recall': 0.8202,
 'precision': 0.8741,
 'min_precision': 0.6912,
 'specificity': 0.7357,
 'f_score': 0.8463}